In [4]:
from datetime import datetime, timedelta
import random

class ClimateFileBase:
    def __init__(self, file_name, start_year, end_year, lat, lon, elev):
        self.file_name = file_name
        self.start_year = start_year
        self.end_year = end_year
        self.lat = lat
        self.lon = lon
        self.elev = elev
        self.header = self.generate_header()
        
    def generate_header(self):
        """Generate metadata header."""
        num_years = self.end_year - self.start_year + 1
        header = [
            f"{self.file_name}: Climate data - file written by SWAT+ editor",
            "nbyr     tstep       lat       lon      elev",
            f"  {num_years}         0    {self.lat:.3f}   {self.lon:.3f}   {self.elev:.3f}"
        ]
        return header
    
    def generate_date_range(self):
        """Generate a date range for the file."""
        start_date = datetime(self.start_year, 1, 1)
        end_date = datetime(self.end_year, 12, 31)
        return start_date, end_date
    
    def write_file(self, records):
        """Write header and data to the file."""
        with open(self.file_name, 'w') as f:
            for line in self.header:
                f.write(line + "\n")
            for record in records:
                f.write(record + "\n")


class PCPFile(ClimateFileBase):
    def generate_records(self):
        """Generate daily precipitation records."""
        start_date, end_date = self.generate_date_range()
        records = []
        current_date = start_date
        while current_date <= end_date:
            year = current_date.year
            day_of_year = current_date.timetuple().tm_yday
            precipitation = f"{random.uniform(0, 50):.5f}"  # Replace with actual data
            records.append(f"{year:4d}  {day_of_year:3d}  {precipitation:>10}")
            current_date += timedelta(days=1)
        return records

    def save(self):
        """Generate and save the .pcp file."""
        records = self.generate_records()
        self.write_file(records)


class TMPFile(ClimateFileBase):
    def generate_records(self):
        """Generate daily temperature records (max and min)."""
        start_date, end_date = self.generate_date_range()
        records = []
        current_date = start_date
        while current_date <= end_date:
            year = current_date.year
            day_of_year = current_date.timetuple().tm_yday
            max_temp = f"{random.uniform(-10, 35):.5f}"  # Replace with actual data
            min_temp = f"{random.uniform(-15, 30):.5f}"  # Replace with actual data
            records.append(f"{year:4d}  {day_of_year:3d}  {max_temp:>10}  {min_temp:>10}")
            current_date += timedelta(days=1)
        return records

    def save(self):
        """Generate and save the .tmp file."""
        records = self.generate_records()
        self.write_file(records)


# Example usage
pcp_file = PCPFile("pcp145.pcp", 1988, 1988, lat=31.727, lon=-83.738, elev=137.465)
pcp_file.save()

tmp_file = TMPFile("lrew_tmp1.tmp", 1988, 1988, lat=31.494, lon=-83.526, elev=118.000)
tmp_file.save()


In [ ]:
import rasterio
import numpy as np
import pandas as pd
from collections import Counter

def read_raster_data(raster_path):
    """Reads raster data and returns the data array."""
    with rasterio.open(raster_path) as src:
        data = src.read(1)
    return data

def calculate_hru_combinations(land_use_data, soil_data, slope_data, area_threshold=1.0):
    """Calculates unique HRU combinations of land use, soil, and slope."""
    hru_array = np.stack((land_use_data, soil_data, slope_data), axis=-1)
    unique_combinations, counts = np.unique(hru_array.reshape(-1, 3), axis=0, return_counts=True)
    
    # Calculate total area (in cells) and determine each HRU's area percentage
    total_area = hru_array.shape[0] * hru_array.shape[1]
    hru_area_percentage = (counts / total_area) * 100
    
    # Filter combinations based on area threshold
    valid_hru_indices = hru_area_percentage >= area_threshold
    valid_combinations = unique_combinations[valid_hru_indices]
    valid_counts = counts[valid_hru_indices]
    valid_areas = hru_area_percentage[valid_hru_indices]
    
    return valid_combinations, valid_counts, valid_areas

def save_hru_data(hru_combinations, counts, areas, output_file):
    """Saves HRU data to a CSV file."""
    hru_data = {
        'HRU_ID': range(1, len(hru_combinations) + 1),
        'Land_Use': hru_combinations[:, 0],
        'Soil': hru_combinations[:, 1],
        'Slope': hru_combinations[:, 2],
        'Cell_Count': counts,
        'Area_Percentage': areas
    }
    hru_df = pd.DataFrame(hru_data)
    hru_df.to_csv(output_file, index=False)
    print(f"HRU data saved to {output_file}")

# Define file paths for the input rasters
land_use_raster = "path/to/land_use.tif"
soil_raster = "path/to/soil.tif"
slope_raster = "path/to/slope.tif"
output_file = "hru_data.csv"

# Read raster data
land_use_data = read_raster_data(land_use_raster)
soil_data = read_raster_data(soil_raster)
slope_data = read_raster_data(slope_raster)

# Calculate HRU combinations
hru_combinations, counts, areas = calculate_hru_combinations(
    land_use_data, soil_data, slope_data, area_threshold=1.0
)

# Save HRU data to CSV
save_hru_data(hru_combinations, counts, areas, output_file)
